#### 1. 人脸特征提取与融合

##### 1.1 process_face_embeddings 函数

该函数调用模型提取人脸特征，

（1）使用到的模型和关键参数有

- face_helper_1:alignment and landmark detection.
  
    RetinaFace 检测bbox + 5 点 landmarks + 人脸对齐及裁剪为`[512,512]`； BiSeNet 解析19类掩码
- face_helper_2:embedding extraction.

    iResNet提取`[1,512]`的ArcFace 向量
- eva_transform_mean & eva_transform_std: for image normalization before passing to EVA model.

    将待评估图像的分布归一化到EVA-CLIP模型的输入域
- app: Application instance used for face detection.

    InsightFace人脸信息实例

（2）函数的输入为RGB图像、模型、EVA-CLIP参数

（3）函数的输出包括
```
id_cond,  # [通用特征1*512，单位化的CLIP输出语义向量1*768]，(1,1280)
id_vit_hidden,  # CLIP最后一层的潜变量 list(torch.Size([1, 577, 1024]))
return_face_features_image_2,  # 彩色的人脸特征图 torch.Size([1, 3, 512, 512])
face_kps  # 来自InsightFace提取，若失败，则用Helper1(RetinaFace)提取 list(5,2)
```

##### 1.2  Local Face Extracter

一个神经网络，融合身份信息与vit视觉特征，用其训练可学习的query，再通过和FFN调制，提取出与身份一致且具有判别力的局部人脸特征

In [ ]:
def forward(self, id_embeds: torch.Tensor, vit_hidden_states: List[torch.Tensor]) -> torch.Tensor:  # [B,257,1024]共5个
        # Repeat latent queries for the batch size
        latents = self.latents.repeat(id_embeds.size(0), 1, 1)  #[B,32，1024]

        # Map the identity embedding to tokens
        id_embeds = self.id_embedding_mapping(id_embeds)  
        id_embeds = id_embeds.reshape(-1, self.num_id_token, self.vit_dim)  # [B,1280]->[B,5,1024]，将id嵌入放到与vit_hidden同维度

        # Concatenate identity tokens with the latent queries
        latents = torch.cat((latents, id_embeds), dim=1)  # [B,32,1024]+[B,5,1024]  # 同时学习, 可能是attention难收敛.

        # Process each of the num_scale visual feature inputs
        for i in range(self.num_scale):
            vit_feature = getattr(self, f"mapping_{i}")(vit_hidden_states[i])
            ctx_feature = torch.cat((id_embeds, vit_feature), dim=1)    # [B,262,1024]，这是id—embeds和vit—feature的拼接

            # Pass through the PerceiverAttention and ConsisIDFeedForward layers
            for attn, ff in self.layers[i * self.depth : (i + 1) * self.depth]:
                latents = attn(ctx_feature, latents) + latents # q:latents;kv_crop:ctx_feature
                latents = ff(latents) + latents

        # Retain only the query latents
        latents = latents[:, : self.num_queries]
        # Project the latents to the output dimension
        latents = latents @ self.proj_out  # [B, 32, 2048]
        return latents

局部提取器会首先随机初始化一个可学习的latents，形状为`[1, num_queries=32, vit_dim=1024]`。接着，id_embeds通过MLP(`[B, 1280]`-> `[B,5,1024]`)变形，
拼接到latents后面。第二部分ctx_features由id_embeds和vit_feature(`[B, 257, 1024]`)拼接而成，并通过交叉注意力融入latents。网络由10组交替的PerceiverAttention(q=context; k,v=latents)和FFN构成

这一设计可追溯到[Perceiver: General Perception with Iterative Attention](arxiv.org/abs/2103.03206)

#### 2. 管线

##### 2.1 ConsisIDTransformer3DModel

前向双流DiT，以CogVideoX为基础。若为训练模式，在第偶数次经过DiT时，会训练Perceiver_cross_attn。q是hidden_states,kv是valid_face_emb

In [ ]:
        # 3. Transformer blocks
        ca_idx = 0
        for i, block in enumerate(self.transformer_blocks):
            hidden_states, encoder_hidden_states = block(
                hidden_states=hidden_states,
                encoder_hidden_states=encoder_hidden_states,
                temb=emb,
                image_rotary_emb=image_rotary_emb,
                attention_kwargs=attention_kwargs,
            )

            if self.is_train_face:
                if i % self.cross_attn_interval == 0 and valid_face_emb is not None: # 在[0,2,4,...,28]
                    hidden_states = hidden_states + self.local_face_scale * self.perceiver_cross_attention[ca_idx](
                        valid_face_emb, hidden_states  # q: hidden_states ;kv:valid_face_emb
                    )  # torch.Size([2, 32, 2048])  torch.Size([2, 17550, 3072])
                    ca_idx += 1

        hidden_states = self.norm_final(hidden_states)

        # 4. Final block
        hidden_states = self.norm_out(hidden_states, temb=emb)
        hidden_states = self.proj_out(hidden_states)

##### 2.2 数据预处理

###### 2.2.1 id跟踪机制

核心函数`find_max_confidence_bbox`会从视频的全部帧中提取出所有的人物(id)，对每个id，提取其在各帧中置信度最高的一个bbox(可能是face,head或person级别的框)，将id、best_bbox坐标及其所在帧、置信度保存在字典中，供视频分割模型SAM使用。

`video_predictor.add_new_points_or_box()`会启动对某帧中人像的分割，接着`predictor.propagate_in_video()`会将该过程传播到视频各帧。
除人物帧掩码、关键帧及其类型外，程序会额外保存`valid_frame`，即每个对象在这些帧被识别到。

接着，在`dataloader.py`中，`get_valid_segments(valid_frame, tolerance=5)`函数将有效帧合并成若干个列表，这允许模型在一个**相对连续的片段上学习**人脸保持. `generate_frame_indices_for_face(n_frames, sample_stride, valid_frame, tolerance=7, ...)`会选取最长的一个片段，将其截断或扩展到`n_frames`，但可能会带来缓慢的运动片段。

最终，使用键`track_id`跟踪视频中出现的人物。

###### 2.2.2 生成训练数据————get_batch()函数

输入: idx
输出：`<tuple>(pixel_values, text, 'video', video_dir, expand_face_imgs, dense_masks_tensor, selected_frame_index, reserve_face_imgs, original_face_imgs)`

is_train_face 分支

首先会导入4个json，分别是

- `bbox_data`给出track_id、特征、所在帧、bbox的数据

In [ ]:
{
    "1": {
        "frames": [0, 1, 2],
        "face": [
            {"frame_id": 0, "box": {"x1": 100, "y1": 120, "x2": 150, "y2": 170}, "confidence": 0.98},
            {"frame_id": 1, "box": {"x1": 102, "y1": 121, "x2": 152, "y2": 171}, "confidence": 0.97}
        ],
        "head": [
            {"frame_id": 1, "box": {"x1": 90, "y1": 100, "x2": 160, "y2": 180}, "confidence": 0.99},
            {"frame_id": 2, "box": {"x1": 91, "y1": 101, "x2": 161, "y2": 181}, "confidence": 0.95}
        ],
        "person": [
             {"frame_id": 1, "box": {"x1": 50, "y1": 80, "x2": 250, "y2": 480}, "confidence": 0.96}
        ]
    },
}

- `corresponding_data`将原始track_id映射到内部掩码id，形如

In [ ]:
{
    "1": {
        "head": 1,
        "person": 2
    },
    "2": {
        "face": 3
    }
}

- `control_sam2_frame`给出track_id的特征对应的关键帧，例如，id为1的人，其头部置信度最高的框位于第1帧

In [ ]:
{
    "1": {
        "head": 1,
        "person": 1
    },
    "2": {
        "face": 1
    }
}

- `valid_frame`给出成功生成掩码的帧位置

In [ ]:
{
    "1": {
        "head": [1, 2, 3],
        "person": [1, 2, 3, 4]
    },
    "2": {
        "face": [1, 2]
    }
}

接着，将`corresponding_data`保存到`<list>valid_id`中，并随机抽取一个track_id作为训练对象，确定需要抓捕的帧

In [ ]:
# get video
                total_index = list(range(video_num_frames))  # 总帧list
                # valid_id对应的人物(head&face)出现帧list
                batch_index, _ = generate_frame_indices_for_face(self.max_num_frames, self.sample_stride, valid_frame[valid_id],
                                                                          self.miss_tolerance, self.skip_frames_start_percent, self.skip_frames_end_percent,
                                                                          self.skip_frames_start, self.skip_frames_end)

接着，调用select_mask_frames_from_index()函数处理得到

- `selected_frame_index`: 使用其内部select_frames_with_distance_constraint()函数选中的帧，它会从`valid_frame`中选出间隔至少为`min_distance`的`num_frames`个帧, 并且优先选择高置信度的帧。
- `selected_masks_dict` : 选中帧的掩码列表，掩码的生成基于对视频帧做分割得到的二值化图像序列。
- `selected_bboxs_dict` : 选中帧的bbox列表
- `dense_masks_dict` : 全体帧的掩码列表


##### 2.3 训练

首先冻结无关参数。只有局部提取器、交叉注意力块、DiT每次注意力的LoRA权重被解冻，其余部分(文本编码器、VAE、DiT的其他块均被冻结)

In [ ]:
if args.is_train_face:  #训LFE和PCA
        unfreeze_modules = ["local_facial_extractor", "perceiver_cross_attention"]

        for module_name in unfreeze_modules:
            try:
                for param in getattr(transformer, module_name).parameters():
                    param.requires_grad = True
            except AttributeError:
                continue

        if args.is_train_lora:  # 增训lora
            transformer_lora_config = LoraConfig(
                r=args.rank,
                lora_alpha=args.lora_alpha,
                init_lora_weights=True,
                target_modules=["to_k", "to_q", "to_v", "to_out.0"],
                # Need to check 'exclude_modules'
                # exclude_modules=unfreeze_modules,
            )
            transformer.add_adapter(transformer_lora_config)

In [ ]:
# 计算 MSE loss
if args.is_train_face and len(valid_indices) == 0:
    # 无有效样本，直接返回 0
    loss = torch.zeros((), device=model_pred.device, dtype=model_pred.dtype)
else:
    loss = (weights * (model_pred - target) ** 2).reshape(batch_size, -1)

    # 仅在训练人脸且开启 mask 时才做掩膜，再求平均
    if args.is_train_face and args.enable_mask_loss and enable_mask_loss_flag:
        loss = (loss * dense_masks).sum() / dense_masks.sum()
    # 否则直接平均，得到：0(no train face), batch_loss(train face), batch_loss_with_mask(train face+mask+mask_flag)
    else:
        loss = loss.mean(dim=1).mean()

损失函数如上。其中target是无噪声的video_latents，model_pred是根据时间步预测的去噪结果。最后在batch维度平均。

#### 3. 评估

##### 3.1 CLIP得分

使用`cap.set()`跳转到视频文件的指定帧, `cap.read()`提取帧, 将原视频等距采样16帧，保存到一个列表`frames`中，并计算这16张图片关于同一prompt的CLIP得分之平均。

##### 3.2 Arc得分, Cur得分, FID得分

分别衡量ref和视频帧之间在

- ArcFace模型、CurricularFace模型特征空间的余弦相似度
- FID得分衡量InceptionV3特征空间中的距离。